In [ ]:
import numpy as np
import scipy.io as sci
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
# Enter the input: input is the microstructural topology and its corresponding normalized property
topology_train=...
topology_test=...
property_train=...
property_test=...

In [ ]:
# The structure of the feed-forward neural network which is used for solving the forward problem
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(7225, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128,7)

    def forward(self, x):
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net = Net()

In [ ]:
# The loss function and optimizer are defined
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.00001)

In [ ]:
# The training process of the forward network
for i in range(200):
  for j in range(100):
    # forward + backward + optimize
    optimizer.zero_grad()
    o = net(topology_train[j*200:((j+1)*200),:]) # o is the output of the network
    loss=criterion(o,property_train[j*200:((j+1)*200),:])
    loss.backward()
    #print(loss.item())
    optimizer.step()
  o2 = net(topology_test)
  loss2=criterion(o2,property_test)
  print(loss2.item())

In [ ]:
# The structure of the network used for the inverse problem
class Net2(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(7,100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100,7225)

    def forward(self, x):
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net2 = Net2()

In [ ]:
# optimizer and loss function are defined
optimizer = optim.Adam(net2.parameters(), lr=0.00001)
criterion=nn.MSELoss()

In [ ]:
# The training process of the inverse network
for i in range(200):
  for j in range(100):
    # forward + backward + optimize
    optimizer.zero_grad()
    o = net2(property_train[j*200:((j+1)*200),:])
    loss=criterion(net(o),property_train[j*200:((j+1)*200),:])+0.1*criterion(o,topology_train[j*200:((j+1)*200),:])
    loss.backward()
    optimizer.step()
  o2 = net2(property_test)
  loss2=criterion(net(o2),property_test)
  print(loss2.item())